In [1]:
using Finch
using SparseArrays
using Metatheory
using Metatheory.EGraphs
using TermInterface
using PrettyPrinting
include("../src/Galley.jl")
using .Galley

12


In [16]:
A = InputTensor(uniform_fiber([10000, 100], .01))
B = InputTensor(uniform_fiber([100, 10000], .01))
C = InputTensor(uniform_fiber([100, 10000], .01))
D = OutTensor()
E = OutTensor()

f(x,y) = 2*x + 5
declare_binary_operator(f)
verbose = 1
# Sum_i Sum_l B[j,l]*A[i,j]*A[i,j]
D["j", "l"] = ∑("i", A["i", "j"] * B["j","l"] * C["j", "l"])
E["o"] = ∑("m",  D["m", "o"])


output_tensor_1 = galley(E, verbose=verbose, optimize=false)

output_tensor_2 = galley(E, verbose=verbose, global_index_order = 1, optimize=true)

#output_tensor_3 = galley(E, verbose=verbose, global_index_order = 2, optimize=true)

#output_tensor_4 = galley(E, verbose=verbose, global_index_order = 3, optimize=true)

#output_tensor_5 = galley(E, verbose=verbose, global_index_order = 4, optimize=true)

#println("Is Correct?: ", output_tensor_1 == output_tensor_2 && output_tensor_2 == output_tensor_3 && 
#                                output_tensor_3 == output_tensor_4 && output_tensor_4 == output_tensor_5)

Expression: Aggregate(+,["m", "i_1"],
   MapJoin(*,
      MapJoin(*,
         Reorder(
            InputTensor(["i_1", "m"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"]),
         Reorder(
            InputTensor(["m", "o"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"])),
      Reorder(
         InputTensor(["m", "o"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"])))
Time to Execute: 

0.343789972
Optimized Expression: Aggregate(+,["m"],
   MapJoin(*,
      MapJoin(*,
         Reorder(
            InputTensor(["m", "o"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"]),
         Reorder(
            InputTensor(["m", "o"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"])),
      Aggregate(+,["i_1"],
         Reorder(
            InputTensor(["i_1", "m"],FIBER,Any["i_1", "m", "o"]),Any["i_1", "m", "o"]))))
Time to Execute: 0.010166398


SparseHash (0) [1:10000]
├─[96]: 95
├─[166]: 95
│ ⋮
├─[9801]: 77
├─[9831]: 98

In [14]:
is = ["i", "j"]
B = InputTensor(uniform_fiber([1000, 1000, 1000], .001))
D = InputTensor(uniform_fiber([1000, 1000], 0.0001))
C = InputTensor(uniform_fiber([1000, 1000], 0.0001))
MTTKRP = OutTensor()
MTTKRP_hand_opt = OutTensor()

verbose = 1
MTTKRP["k", "l"] = ∑(["i", "j"],  B["i", "k", "l"] * C["k", "j"] * D["l","j"])

output_tensor_1 = galley(MTTKRP, optimize=false, verbose=verbose)

output_tensor_2 = galley(MTTKRP, optimize=true, global_index_order = 5, verbose=verbose)

#MTTKRP_hand_opt["k", "l"] = ∑(["i"],  B["i", "k", "l"]  * ∑(["j"], C["k", "j"] *  D["l","j"]))
#output_tensor_2 = galley(MTTKRP_hand_opt, optimize=false, verbose=verbose)

Expression: Aggregate(+,["i", "j"],
   MapJoin(*,
      MapJoin(*,
         Reorder(
            InputTensor(["i", "k", "l"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"]),
         InputTensor(["k", "j"],FIBER,Any["i", "j", "k", "l"])),
      InputTensor(["l", "j"],FIBER,Any["i", "j", "k", "l"])))
Time to Execute: 

1.228956602
Optimized Expression: Reorder(
   Aggregate(+,["j", "i"],
      MapJoin(*,
         Reorder(
            InputTensor(["l", "j"],FIBER,Any["i", "l", "j", "k"]),Any["i", "l", "j", "k"]),
         MapJoin(*,
            InputTensor(["k", "j"],FIBER,Any["i", "l", "j", "k"]),
            InputTensor(["i", "k", "l"],FIBER,Any["i", "l", "j", "k"])))),["k", "l"])
Time to Execute: 

1.806789343


SparseHash (0) [:,1:1000]
├─[:,101]: SparseHash (0) [1:1000]
│ ├─[731]: 1
├─[:,122]: SparseHash (0) [1:1000]
│ ├─[784]: 1
│ ⋮
├─[:,479]: SparseHash (0) [1:1000]
│ ├─[609]: 2
├─[:,897]: SparseHash (0) [1:1000]
│ ├─[321]: 1

In [4]:
n_1 = 1000
declare_binary_operator(^)
X = InputTensor(uniform_fiber([n_1, n_1, n_1], .0001))
U = InputTensor(uniform_fiber([n_1], 1.0))
V = InputTensor(uniform_fiber([n_1], 1.0))
W = InputTensor(uniform_fiber([n_1], 1.0))
A = OutTensor()
Y = OutTensor()
C = OutTensor()

C["i","j","k"] = U["i"]*V["j"]*W["k"]
Y[] = ∑(["i","j","k"], (X["i","j","k"] + C["i","j","k"])^2)


verbose = 2
# Sum_i Sum_j (X[i,j]+u[i]*v[j])
#output_tensor_1 = galley(Y, verbose=verbose, optimize=false)

# (Sum_i u[i]^2)*(Sum_j v[j]^2) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_[i,j] X[i,j]^2)
output_tensor_2 = galley(Y, verbose=verbose, optimize=true)

#println("Is Correct?: ", output_tensor_1 == output_tensor_2)

SaturationReport
	Stop Reason: saturated
	Iterations: 91
	EGraph Size: 245 eclasses, 4050 nodes
 ────────────────────────────────────────────────────────────────────
                            Time                    Allocations      
                   ───────────────────────   ────────────────────────
 Tot / % measured:      4.41s /  95.3%           1.65GiB /  98.8%    

 Section   ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────
 Apply         91    2.31s   55.1%  25.4ms    678MiB   40.5%  7.45MiB
 Search        91    1.62s   38.6%  17.8ms   0.94GiB   57.8%  10.6MiB
   6           91    832ms   19.8%  9.14ms    500MiB   29.9%  5.50MiB
   5           91    135ms    3.2%  1.48ms   81.9MiB    4.9%   921KiB
   12          91    132ms    3.1%  1.45ms   14.4MiB    0.9%   162KiB
   7           91    117ms    2.8%  1.29ms   66.1MiB    3.9%   744KiB
   4           91    110ms    2.6%  1.21ms   76.0MiB    4.5%   

Time to Execute: 6.328755585


1000299895

In [5]:
is = ["i", "j", "k", "l"]
E_fiber = uniform_fiber([1000, 1000], .01)
E_1 = InputTensor(E_fiber)
E_2 = InputTensor(E_fiber)
E_3 = InputTensor(E_fiber)
E_4 = InputTensor(E_fiber)

verbose = 1
D["i", "j", "k", "l"] = E_1["i","j"] * E_2["j", "k"] * E_3["k", "l"] * E_4["l", "i"]
query =  ∑(is, D["i", "j", "k", "l"])
output_1 = galley(query, verbose=verbose, optimize = false)
output_2 = galley(query, verbose=verbose, optimize = true)

Expression: Aggregate(+,["i", "j", "k", "l"],
   MapJoin(*,
      MapJoin(*,
         MapJoin(*,
            Reorder(
               InputTensor(["i", "j"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"]),
            Reorder(
               InputTensor(["j", "k"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"])),
         Reorder(
            InputTensor(["k", "l"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"])),
      InputTensor(["l", "i"],FIBER,Any["i", "j", "k", "l"])))
Time to Execute: 

4.725157194


Optimized Expression: Aggregate(+,["i", "j", "k", "l"],
   MapJoin(*,
      MapJoin(*,
         Reorder(
            InputTensor(["k", "l"],FIBER,["i", "j", "k", "l"]),["i", "j", "k", "l"]),
         InputTensor(["l", "i"],FIBER,["i", "j", "k", "l"])),
      MapJoin(*,
         Reorder(
            InputTensor(["j", "k"],FIBER,["i", "j", "k", "l"]),["i", "j", "k", "l"]),
         Reorder(
            InputTensor(["i", "j"],FIBER,["i", "j", "k", "l"]),["i", "j", "k", "l"]))))
Time to Execute: 

2.709829752


10276

In [6]:
# Suppose you have a network of power nodes and lines. Further, you need to take weather events/outages into account.
# The following query provides the total available power at each node.
num_nodes = 1000
height = 100
width = 100

# Because we're dealing with positive numbers, the following is true.
declare_binary_operator(min)

transmission_capacity = InputTensor(uniform_fiber([num_nodes, num_nodes], 5.0/num_nodes))
node_location = InputTensor(uniform_fiber([num_nodes, height, width], 1.0/height/width))
weather_event = InputTensor(uniform_fiber([height, width], .01, default_value = 1, non_default_value = 0))
node_outage_percent = InputTensor(uniform_fiber([num_nodes], .1, default_value = 1, non_default_value = 0))
node_capacity = InputTensor(uniform_fiber([num_nodes], 1.0))
node_generation = OutTensor()
available_power = OutTensor()

node_generation["node_id"] = ∑(["lat", "long"], node_location["node_id", "lat", "long"] * weather_event["lat", "long"] * node_outage_percent["node_id"] * node_capacity["node_id"])
available_power["node_id1"] = node_generation["node_id1"] + ∑("node_id2", min(transmission_capacity["node_id1", "node_id2"], node_generation["node_id2"]))


verbose = 1
total_neighbor_power_1 = galley(available_power, verbose=verbose, optimize=false)

total_neighbor_power_2 = galley(available_power, verbose=verbose, optimize=true)

println("Is Equivalent: ", total_neighbor_power_1 == total_neighbor_power_2)

Expression: MapJoin(+,
   Aggregate(+,["lat_1", "long_1"],
      MapJoin(*,
         MapJoin(*,
            MapJoin(*,
               InputTensor(["node_id1", "lat_1", "long_1"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),
               Reorder(
                  InputTensor(["lat_1", "long_1"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"])),
            Reorder(
               InputTensor(["node_id1"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"])),
         Reorder(
            InputTensor(["node_id1"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]))),
   Aggregate(+,["node_id2"],
      MapJoin(min,
         Reorder(
            InputTensor(["node_id1", "node_id2"],FIBER,Any["lat_1

10.792913229
Optimized Expression: MapJoin(+,
   Aggregate(+,["node_id2"],
      MapJoin(min,
         Reorder(
            InputTensor(["node_id1", "node_id2"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),
         MapJoin(*,
            Reorder(
               InputTensor(["node_id2"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),
            MapJoin(*,
               Reorder(
                  InputTensor(["node_id2"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),
               Aggregate(+,["lat_2", "long_2"],
                  MapJoin(*,
                     Reorder(
                        InputTensor(["lat_2", "long_2"],FIBER,Any["lat_1", "lat_2", "long_1", "long_2", "node_id1", "node_id2"]),Any["lat_

4.253129131


Is Equivalent: true
